# Self_Driving_Car

*  Credits: https://github.com/SullyChen/Autopilot-TensorFlow
*  Research paper: End to End Learning for Self-Driving Cars by Nvidia. [https://arxiv.org/pdf/1604.07316.pdf]

*  NVidia dataset: 72 hrs of video => 72*60*60*30 = 7,776,000 images
*  Nvidia blog: https://devblogs.nvidia.com/deep-learning-self-driving-cars/


*  Our Dataset: https://github.com/SullyChen/Autopilot-TensorFlow [https://drive.google.com/file/d/0B-KJCaaF7elleG1RbzVPZWV4Tlk/view]
*  Size: 25 minutes = 25*60*30 = 45,000 images ~ 2.3 GB


*  If you want to try on a slightly large dataset: 70 minutes of data ~ 223GB
*  Refer: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5
*  Format: Image, latitude, longitude, gear, brake, throttle, steering angles and speed



* Additional Installations:
*  pip3 install h5py


*  AWS: https://aws.amazon.com/blogs/machine-learning/get-started-with-deep-learning-using-the-aws-deep-learning-ami/

*  Youtube:https://www.youtube.com/watch?v=qhUvQiKec2U
*  Further reading and extensions: https://medium.com/udacity/teaching-a-machine-to-steer-a-car-d73217f2492c
*  More data: https://medium.com/udacity/open-sourcing-223gb-of-mountain-view-driving-data-f6b5593fbfa5

#### Our Dataset: https://github.com/SullyChen/Autopilot-TensorFlow [https://drive.google.com/file/d/0B-KJCaaF7elleG1RbzVPZWV4Tlk/view]
### Size: 25 minutes = 25*60*30 = 45,000 images ~ 2.3 GB


## 1. Selp_driving_car Exploratory Data Analysis

### Importing bunch of necessary libraries

In [2]:

from __future__ import division

import os
import numpy as np
import random

from scipy import pi
from itertools import islice
# pip3 install h5py
#pip3 install opencv-python

import scipy.misc
import model
import cv2
from subprocess import call
import math
import tensorflow as tf
import scipy
import warnings
warnings.filterwarnings('ignore')
import os
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

/home/nagesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nagesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nagesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nagesh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [3]:
# read images and steering angles from driving_dataset folder

DATA_FOLDER = './driving_dataset/' # change this to your folder
TRAIN_FILE = os.path.join(DATA_FOLDER, 'data.txt')  #join the data.txt file to the driving_dataset


split =0.8  #split the data train data = 80%
X = []   
y = []
LIMIT = None
# open the train file which is driving dataset  with data.txt files
with open(TRAIN_FILE) as fp:
    for line in islice(fp,LIMIT):
        path, angle = line.strip().split()   #in data.txt we have each image corresponding with each angle so each line split into path and angle 
        full_path = os.path.join(DATA_FOLDER, path)
        X.append(full_path)  # now X's are onli images ID's 
        
        # converting angle from degrees to radians
        y.append(float(angle) * pi / 180 )   # now yi's are only  radins(actual degrees converted into radians)


y = np.array(y)     # alll radiens are in array 
print("Completed processing data.txt")

split_index = int(len(y)*0.8)   # split the yi's train dataset = 80%

train_y = y[:split_index]   # train data set = 80%

test_y = y[split_index:]   #test data = 20%

Completed processing data.txt


In [4]:
%matplotlib
import numpy;
import warnings
warnings.filterwarnings('ignore')

# PDF of train and test 'y' values. 
import matplotlib.pyplot as plt 
# plot histogram  for train angle (radians) and test angle(radiens)

plt.hist(train_y, bins=50, normed=1, color='green', histtype ='step');
plt.hist(test_y, bins=50, normed=1, color='red', histtype ='step');

plt.show()

Using matplotlib backend: GTK3Agg


* here green histogram plot implies train data and  red histogram plot implies test data 
* we can observe most often yi's are  0's
* angles(degrees) converted into radiens

In [5]:
#Model 0: Base line Model: y_test_pred = mean(y_train_i) 
train_mean_y = np.mean(train_y)

print('Test_MSE(MEAN):%f' % np.mean(np.square(test_y-train_mean_y)) )

print('Test_MSE(ZERO):%f' % np.mean(np.square(test_y-0.0)) )

Test_MSE(MEAN):0.191127
Test_MSE(ZERO):0.190891


* mean value of yi's(mostly yi's are 0 degrees)  so  most of predicted yi^ test yi's are same
* so that mean squered error  MSE= 1/n under root test yi's - train yi's
          * then we got mse value around 0.19 

## 2. Driving Data  (batch wise load the data)

### 1. Load the Data

* from diving_dataset images we have one text file with each image id and with angle representation for each image 

In [6]:
xs = []
ys = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0

#read data.txt
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])    #all image id's are append to empy list
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)  # all angle values are append to ys list after angles converted into radans


* now xs are all sequence images ids
* now ys are all radians(all angles converted into radians)

### 2.  Split the data into Train and Test/Validation

as per assignment we have to split our data train data = 70% and test/validation data=30%

In [7]:
#get number of images
num_images = len(xs)    

print(num_images)

45406


In [8]:
#get number of images
num_images = len(xs)    

# split 70% of train data and 30% of test data

train_xs = xs[:int(len(xs) * 0.7)]
train_ys = ys[:int(len(xs) * 0.7)]

val_xs = xs[-int(len(xs) * 0.3):]
val_ys = ys[-int(len(xs) * 0.3):]

num_train_images = len(train_xs)
num_val_images = len(val_xs)


In [9]:
print("train images:",num_train_images)
print("test images:",num_val_images)

train images: 36324
test images: 9081


In [10]:
def LoadTrainBatch(batch_size):
    global train_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(train_xs[(train_batch_pointer + i) % num_train_images])[-150:], [66, 200]) / 255.0)
        y_out.append([train_ys[(train_batch_pointer + i) % num_train_images]])
    train_batch_pointer += batch_size
    return x_out, y_out

def LoadValBatch(batch_size):
    global val_batch_pointer
    x_out = []
    y_out = []
    for i in range(0, batch_size):
        x_out.append(scipy.misc.imresize(scipy.misc.imread(val_xs[(val_batch_pointer + i) % num_val_images])[-150:], [66, 200]) / 255.0)
        y_out.append([val_ys[(val_batch_pointer + i) % num_val_images]])
    val_batch_pointer += batch_size
    return x_out, y_out


* we are selecting batch wise images  from train data and validation data

## 3.Model

In [11]:
# function for weights 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W, stride):
  return tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')


In [12]:
x = tf.placeholder(tf.float32, shape=[None, 66, 200, 3])
y_ = tf.placeholder(tf.float32, shape=[None, 1])

In [13]:
x_image = x

#first convolutional layer
W_conv1 = weight_variable([5, 5, 3, 24])
b_conv1 = bias_variable([24])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 2) + b_conv1)

#second convolutional layer
W_conv2 = weight_variable([5, 5, 24, 36])
b_conv2 = bias_variable([36])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, 2) + b_conv2)

#third convolutional layer
W_conv3 = weight_variable([5, 5, 36, 48])
b_conv3 = bias_variable([48])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 2) + b_conv3)

#fourth convolutional layer
W_conv4 = weight_variable([3, 3, 48, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, 1) + b_conv4)


#fifth convolutional layer
W_conv5 = weight_variable([3, 3, 64, 64])
b_conv5 = bias_variable([64])

h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5, 1) + b_conv5)

#FCL 1
W_fc1 = weight_variable([1152, 1164])
b_fc1 = bias_variable([1164])

h_conv5_flat = tf.reshape(h_conv5, [-1, 1152])
h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#FCL 2
W_fc2 = weight_variable([1164, 100])
b_fc2 = bias_variable([100])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

#FCL 3
W_fc3 = weight_variable([100, 50])
b_fc3 = bias_variable([50])

h_fc3 = tf.nn.relu(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)

h_fc3_drop = tf.nn.dropout(h_fc3, keep_prob)

#FCL 3
W_fc4 = weight_variable([50, 10])
b_fc4 = bias_variable([10])

h_fc4 = tf.nn.relu(tf.matmul(h_fc3_drop, W_fc4) + b_fc4)

h_fc4_drop = tf.nn.dropout(h_fc4, keep_prob)

#Output
W_fc5 = weight_variable([10, 1])
b_fc5 = bias_variable([1])

y = tf.multiply(tf.atan(tf.matmul(h_fc4_drop, W_fc5) + b_fc5), 2) #scale the output



* we have 5 convlution networks

# 4. train the model

In [14]:
# file directory
LOGDIR = './save'

In [15]:
sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())


W1002 07:36:34.192246 139821803878208 deprecation.py:323] From /home/nagesh/.local/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [16]:
# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())


In [17]:
epochs = 30  
batch_size = 100


In [18]:
# train over the dataset about 30 times
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)  #load batch wise train  data 
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})  #selection dropout rate = 0.8
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)  #load batchwise validation data
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 1.0})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)

    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model2.ckpt")   #after train the model then save in the save folder
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")


W1002 07:36:40.963131 139821803878208 saver.py:1134] *******************************************************
W1002 07:36:40.964240 139821803878208 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1002 07:36:40.965145 139821803878208 saver.py:1136] Consider switching to the more efficient V2 format:
W1002 07:36:40.966065 139821803878208 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1002 07:36:40.966883 139821803878208 saver.py:1138] now on by default.
W1002 07:36:40.967869 139821803878208 saver.py:1139] *******************************************************


Epoch: 0, Step: 0, Loss: 14.2811
Epoch: 0, Step: 10, Loss: 12.202
Epoch: 0, Step: 20, Loss: 12.0212
Epoch: 0, Step: 30, Loss: 11.8651
Epoch: 0, Step: 40, Loss: 11.6876
Epoch: 0, Step: 50, Loss: 11.5989
Epoch: 0, Step: 60, Loss: 11.5508
Epoch: 0, Step: 70, Loss: 12.3692
Epoch: 0, Step: 80, Loss: 13.0537
Epoch: 0, Step: 90, Loss: 11.2532


W1002 07:38:42.049209 139821803878208 saver.py:1134] *******************************************************
W1002 07:38:42.050610 139821803878208 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1002 07:38:42.051404 139821803878208 saver.py:1136] Consider switching to the more efficient V2 format:
W1002 07:38:42.052261 139821803878208 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1002 07:38:42.053132 139821803878208 saver.py:1138] now on by default.
W1002 07:38:42.053979 139821803878208 saver.py:1139] *******************************************************


Epoch: 0, Step: 100, Loss: 10.8229
Epoch: 0, Step: 110, Loss: 10.6581
Epoch: 0, Step: 120, Loss: 10.5112
Epoch: 0, Step: 130, Loss: 10.3864
Epoch: 0, Step: 140, Loss: 10.285
Epoch: 0, Step: 150, Loss: 10.1952
Epoch: 0, Step: 160, Loss: 10.0655
Epoch: 0, Step: 170, Loss: 9.87761
Epoch: 0, Step: 180, Loss: 9.80307
Epoch: 0, Step: 190, Loss: 9.97174


W1002 07:40:28.677311 139821803878208 saver.py:1134] *******************************************************
W1002 07:40:28.678476 139821803878208 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1002 07:40:28.679302 139821803878208 saver.py:1136] Consider switching to the more efficient V2 format:
W1002 07:40:28.680051 139821803878208 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1002 07:40:28.680898 139821803878208 saver.py:1138] now on by default.
W1002 07:40:28.681794 139821803878208 saver.py:1139] *******************************************************


...

Epoch: 29, Step: 3100, Loss: 0.387359
Epoch: 29, Step: 3110, Loss: 0.389441
Epoch: 29, Step: 3120, Loss: 0.288675
Epoch: 29, Step: 3130, Loss: 0.289041
Epoch: 29, Step: 3140, Loss: 0.337495
Epoch: 29, Step: 3150, Loss: 0.380643
Epoch: 29, Step: 3160, Loss: 0.312784
Epoch: 29, Step: 3170, Loss: 0.308037
Epoch: 29, Step: 3180, Loss: 0.399987
Epoch: 29, Step: 3190, Loss: 0.940393


W1002 11:15:29.367220 139821803878208 saver.py:1134] *******************************************************
W1002 11:15:29.368461 139821803878208 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1002 11:15:29.369419 139821803878208 saver.py:1136] Consider switching to the more efficient V2 format:
W1002 11:15:29.370631 139821803878208 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1002 11:15:29.371781 139821803878208 saver.py:1138] now on by default.
W1002 11:15:29.373514 139821803878208 saver.py:1139] *******************************************************


Epoch: 29, Step: 3200, Loss: 1.25355
Epoch: 29, Step: 3210, Loss: 0.646579
Epoch: 29, Step: 3220, Loss: 0.915587
Epoch: 29, Step: 3230, Loss: 0.27994
Epoch: 29, Step: 3240, Loss: 0.317431
Epoch: 29, Step: 3250, Loss: 0.491586
Epoch: 29, Step: 3260, Loss: 0.603905
Epoch: 29, Step: 3270, Loss: 0.467377
Epoch: 29, Step: 3280, Loss: 0.362398
Epoch: 29, Step: 3290, Loss: 0.364944


W1002 11:17:12.244411 139821803878208 saver.py:1134] *******************************************************
W1002 11:17:12.245638 139821803878208 saver.py:1135] TensorFlow's V1 checkpoint format has been deprecated.
W1002 11:17:12.246744 139821803878208 saver.py:1136] Consider switching to the more efficient V2 format:
W1002 11:17:12.247641 139821803878208 saver.py:1137]    `tf.train.Saver(write_version=tf.train.SaverDef.V2)`
W1002 11:17:12.248493 139821803878208 saver.py:1138] now on by default.
W1002 11:17:12.249402 139821803878208 saver.py:1139] *******************************************************


Epoch: 29, Step: 3300, Loss: 0.274069
Epoch: 29, Step: 3310, Loss: 0.27062
Epoch: 29, Step: 3320, Loss: 0.279923
Epoch: 29, Step: 3330, Loss: 0.279385
Epoch: 29, Step: 3340, Loss: 0.317284
Epoch: 29, Step: 3350, Loss: 0.285902
Model saved in file: ./save/model2.ckpt
Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser


# run the model

* after we have  train the model then we store that model in save folder as model.ckpt formate

In [3]:
#InteractiveSession is that InteractiveSession makes itself the default session so that you can call run() or eval() without explicitly calling the session: https://stackoverflow.com/questions/41791469/whats-the-difference-between-tf-session-and-tf-interactivesession
sess = tf.InteractiveSession()
# save the train  model 
saver = tf.train.Saver()
saver.restore(sess, "save/model2.ckpt")

#read the steering wheel image
img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape

smoothed_angle = 0



W1002 18:18:45.571818 140089816483648 deprecation.py:323] From /home/nagesh/.local/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [4]:
#read data.txt
xs = []  #assume all xs are empty list 
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])   #empty list append with each  datapoints in driving dataset 
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        ys.append(float(line.split()[1]) * scipy.pi / 180)

#get number of images
num_images = len(xs)


i = math.ceil(num_images*0.8)
print("Starting frameofvideo:" +str(i))


Starting frameofvideo:36325


In [6]:
#if we enter q button then program execution will be stops
while(cv2.waitKey(10) != ord('q')):
    # read the image
    full_image = scipy.misc.imread("driving_dataset/" + str(i) + ".jpg", mode="RGB")
    # image resize into  66x200 then each pixel value   divide by 255.0
    image = scipy.misc.imresize(full_image[-150:], [66, 200]) / 255.0
    #degrees convert into radians
    degrees = model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0] * 180.0 / scipy.pi
    #call("clear")
    #print("Predicted Steering angle: " + str(degrees))
    print("Steering angle: " + str(degrees) + " (pred)\t" + str(ys[i]*180/scipy.pi) + " (actual)")
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    #make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    #and the predicted angle
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    # smoothed angle  execute o steering wheel image
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()


Steering angle: 75.4582155728192 (pred)	48.5 (actual)
Steering angle: 56.278696043775724 (pred)	48.71000000000001 (actual)
Steering angle: 63.4811919952567 (pred)	48.91 (actual)
Steering angle: 58.979427973679634 (pred)	49.11 (actual)
Steering angle: 61.178858169450464 (pred)	49.21 (actual)
Steering angle: 69.72815131204206 (pred)	49.41 (actual)
Steering angle: 77.93889246804365 (pred)	49.92 (actual)
Steering angle: 89.73113210780062 (pred)	50.12 (actual)
Steering angle: 84.84281353014669 (pred)	50.22 (actual)
Steering angle: 75.67412468267247 (pred)	50.32 (actual)
Steering angle: 79.24654584502092 (pred)	50.32 (actual)
Steering angle: 75.60181346992955 (pred)	50.32 (actual)
Steering angle: 72.16287469453066 (pred)	50.52 (actual)
Steering angle: 68.3051292097941 (pred)	50.92 (actual)
Steering angle: 68.33522302327717 (pred)	51.23 (actual)
Steering angle: 68.67146640592773 (pred)	51.330000000000005 (actual)
Steering angle: 66.42549530115244 (pred)	51.330000000000005 (actual)
Steering an

Steering angle: 9.530363585725661 (pred)	25.820000000000004 (actual)
Steering angle: 8.47097477847365 (pred)	26.12 (actual)
Steering angle: 7.157737294963059 (pred)	26.520000000000003 (actual)
Steering angle: 5.511157551651875 (pred)	26.82 (actual)
Steering angle: 5.562807442155512 (pred)	27.03 (actual)
Steering angle: 5.04402682704954 (pred)	27.43 (actual)
Steering angle: 5.340271341260033 (pred)	27.73 (actual)
Steering angle: 5.987361328792925 (pred)	28.13 (actual)
Steering angle: 6.944217897545644 (pred)	28.44 (actual)
Steering angle: 6.9524303462489385 (pred)	28.840000000000003 (actual)
Steering angle: 7.014250815313547 (pred)	29.04 (actual)
Steering angle: 7.471696758559627 (pred)	29.450000000000003 (actual)
Steering angle: 8.238680898575161 (pred)	29.95 (actual)
Steering angle: 7.957450420820826 (pred)	30.149999999999995 (actual)
Steering angle: 8.236727464472539 (pred)	30.25 (actual)
Steering angle: 9.45836826923185 (pred)	30.449999999999992 (actual)
Steering angle: 8.6169239682

Steering angle: 7.153692542313943 (pred)	18.05 (actual)
Steering angle: 7.366816642533143 (pred)	18.05 (actual)
Steering angle: 7.215580886383428 (pred)	18.05 (actual)
Steering angle: 7.018171770783957 (pred)	18.05 (actual)
Steering angle: 7.022325379572972 (pred)	17.95 (actual)
Steering angle: 7.251866266349121 (pred)	17.75 (actual)
Steering angle: 7.156746490646585 (pred)	17.04 (actual)
Steering angle: 6.353158086208483 (pred)	16.03 (actual)
Steering angle: 5.241272871882576 (pred)	15.230000000000002 (actual)
Steering angle: 4.728154483598545 (pred)	14.32 (actual)
Steering angle: 3.815405142071653 (pred)	13.92 (actual)
Steering angle: 2.8115030840895323 (pred)	13.82 (actual)
Steering angle: 2.212610753321658 (pred)	13.82 (actual)
Steering angle: 1.2817066622591664 (pred)	13.610000000000001 (actual)
Steering angle: 1.0796792715637096 (pred)	13.31 (actual)
Steering angle: 0.761148490421699 (pred)	12.910000000000002 (actual)
Steering angle: 2.348581250119806 (pred)	12.5 (actual)
Steerin

Steering angle: -4.837136981995268 (pred)	-3.4300000000000006 (actual)
Steering angle: -4.419707409549001 (pred)	-3.4300000000000006 (actual)
Steering angle: -3.6694777235675073 (pred)	-4.13 (actual)
Steering angle: -3.7830065666305206 (pred)	-4.44 (actual)
Steering angle: -3.918542706086138 (pred)	-4.54 (actual)
Steering angle: -4.071430087354527 (pred)	-4.74 (actual)
Steering angle: -4.706273972592409 (pred)	-5.75 (actual)
Steering angle: -3.4028028058212914 (pred)	-6.76 (actual)
Steering angle: -2.503865600900046 (pred)	-7.760000000000002 (actual)
Steering angle: -1.0690147849483809 (pred)	-7.969999999999999 (actual)
Steering angle: -1.0676395690476812 (pred)	-7.969999999999999 (actual)
Steering angle: -1.1221377551569187 (pred)	-7.969999999999999 (actual)
Steering angle: -0.2677224446493138 (pred)	-8.27 (actual)
Steering angle: 0.6819837698160479 (pred)	-8.77 (actual)
Steering angle: 1.2486466256856006 (pred)	-9.08 (actual)
Steering angle: -0.4137497279896443 (pred)	-9.28 (actual)


Steering angle: -8.822877223570107 (pred)	-19.56 (actual)
Steering angle: -9.356083625089969 (pred)	-19.46 (actual)
Steering angle: -8.850198834023805 (pred)	-19.46 (actual)
Steering angle: -8.538509981439443 (pred)	-19.46 (actual)
Steering angle: -8.475941179773894 (pred)	-19.46 (actual)
Steering angle: -9.43777098001603 (pred)	-19.36 (actual)
Steering angle: -10.643855175166989 (pred)	-19.36 (actual)
Steering angle: -11.510998916718883 (pred)	-19.36 (actual)
Steering angle: -12.95053641637994 (pred)	-19.87 (actual)
Steering angle: -13.469656406510298 (pred)	-20.97 (actual)
Steering angle: -15.6086078385016 (pred)	-22.29 (actual)
Steering angle: -17.670414845138545 (pred)	-24.200000000000003 (actual)
Steering angle: -18.58301178806958 (pred)	-25.61 (actual)
Steering angle: -14.480863351809747 (pred)	-27.43 (actual)
Steering angle: -14.273633704844267 (pred)	-29.14 (actual)
Steering angle: -15.029716862944461 (pred)	-30.449999999999992 (actual)
Steering angle: -15.720209714445023 (pred

...

Steering angle: 4.438151054742151 (pred)	4.24 (actual)
Steering angle: 4.454198584197095 (pred)	4.24 (actual)
Steering angle: 4.364633459837072 (pred)	4.13 (actual)
Steering angle: 4.122925444828192 (pred)	4.34 (actual)
Steering angle: 3.908821639349917 (pred)	4.74 (actual)
Steering angle: 3.929656277639395 (pred)	5.24 (actual)
Steering angle: 4.021001946938676 (pred)	5.75 (actual)
Steering angle: 4.024395270295703 (pred)	5.75 (actual)
Steering angle: 4.0182656024024395 (pred)	5.849999999999999 (actual)
Steering angle: 4.128229940492353 (pred)	5.849999999999999 (actual)
Steering angle: 4.089923251419159 (pred)	5.849999999999999 (actual)
Steering angle: 4.234415903310778 (pred)	5.55 (actual)
Steering angle: 4.266903698097945 (pred)	5.24 (actual)
Steering angle: 4.496424094306583 (pred)	4.84 (actual)
Steering angle: 4.370159082875612 (pred)	4.84 (actual)
Steering angle: 4.368563379930768 (pred)	4.84 (actual)
Steering angle: 4.465383018962991 (pred)	4.74 (actual)
Steering angle: 4.5869988

Steering angle: 18.041835409561962 (pred)	1.71 (actual)
Steering angle: 50.94920530234691 (pred)	1.41 (actual)
Steering angle: 39.84368312702677 (pred)	1.21 (actual)
Steering angle: 34.81449189774133 (pred)	1.21 (actual)
Steering angle: 16.75737393991146 (pred)	1.21 (actual)
Steering angle: 6.168239679053673 (pred)	0.0 (actual)
Steering angle: 4.064362975997679 (pred)	0.91 (actual)
Steering angle: 3.442478520936249 (pred)	0.61 (actual)
Steering angle: 4.9590029247139285 (pred)	0.4 (actual)
Steering angle: 5.05479120518148 (pred)	0.4 (actual)
Steering angle: 4.833090948685682 (pred)	0.4 (actual)
Steering angle: 4.60032663950811 (pred)	1.41 (actual)
Steering angle: 4.3455029537455125 (pred)	2.32 (actual)
Steering angle: 4.002440480982343 (pred)	3.23 (actual)
Steering angle: 3.9146174817474964 (pred)	3.9300000000000006 (actual)
Steering angle: 3.5588241767016577 (pred)	4.13 (actual)
Steering angle: 3.726900404580274 (pred)	4.24 (actual)
Steering angle: 3.470377922705047 (pred)	4.24 (actua

Steering angle: 1.9301988662843212 (pred)	-4.44 (actual)
Steering angle: 2.354087876694717 (pred)	-3.4300000000000006 (actual)
Steering angle: 2.1288282378676966 (pred)	-2.7200000000000006 (actual)
Steering angle: 2.8604887739300406 (pred)	-2.12 (actual)
Steering angle: 2.8547932499358963 (pred)	-1.51 (actual)
Steering angle: 2.6548309242473107 (pred)	-0.81 (actual)
Steering angle: 2.345115142559436 (pred)	0.2 (actual)
Steering angle: 2.7315429132498394 (pred)	1.11 (actual)
Steering angle: 3.293601954814539 (pred)	1.51 (actual)
Steering angle: 2.5829431832329264 (pred)	1.6100000000000003 (actual)
Steering angle: 2.3372209379827824 (pred)	1.82 (actual)
Steering angle: 1.8827326800907291 (pred)	2.12 (actual)
Steering angle: 1.7150882482337049 (pred)	2.42 (actual)
Steering angle: 1.8020903441076812 (pred)	2.7200000000000006 (actual)
Steering angle: 0.9114362376589571 (pred)	2.92 (actual)
Steering angle: 1.3075086623442416 (pred)	2.92 (actual)
Steering angle: 1.034829154543938 (pred)	3.03 

Steering angle: 6.212682012435653 (pred)	12.81 (actual)
Steering angle: 5.917833418136781 (pred)	12.709999999999999 (actual)
Steering angle: 5.931790055932056 (pred)	12.61 (actual)
Steering angle: 5.977806747917724 (pred)	12.4 (actual)
Steering angle: 4.992304792672795 (pred)	12.2 (actual)
Steering angle: 4.839498946787623 (pred)	11.5 (actual)
Steering angle: 4.354358720891257 (pred)	10.389999999999999 (actual)
Steering angle: 4.503807101912544 (pred)	9.68 (actual)
Steering angle: 4.362211730889479 (pred)	9.380000000000003 (actual)
Steering angle: 4.604759432279448 (pred)	8.77 (actual)
Steering angle: 4.836485979590002 (pred)	8.37 (actual)
Steering angle: 4.847062100632943 (pred)	7.969999999999999 (actual)
Steering angle: 4.703107326138462 (pred)	7.559999999999999 (actual)
Steering angle: 4.655113721274898 (pred)	7.06 (actual)
Steering angle: 4.687801299095287 (pred)	6.25 (actual)
Steering angle: 4.746444876422193 (pred)	5.340000000000001 (actual)
Steering angle: 4.52632538183243 (pred

Steering angle: 0.44735377845843105 (pred)	-8.87 (actual)
Steering angle: 0.5820312617497574 (pred)	-8.87 (actual)
Steering angle: 0.17314089318945974 (pred)	-8.87 (actual)
Steering angle: 0.0509147880591605 (pred)	-8.87 (actual)
Steering angle: 0.23763388721024734 (pred)	-8.87 (actual)
Steering angle: -0.08192810575408972 (pred)	-8.87 (actual)
Steering angle: 0.14879388348015593 (pred)	-8.87 (actual)
Steering angle: 1.0977374379555769 (pred)	-8.97 (actual)
Steering angle: 1.026184589653935 (pred)	-8.97 (actual)
Steering angle: 0.6514905503490847 (pred)	0.0 (actual)
Steering angle: 1.0985732823552572 (pred)	-9.380000000000003 (actual)
Steering angle: 0.7861518386215587 (pred)	-9.68 (actual)
Steering angle: 1.0497382835633124 (pred)	-9.88 (actual)
Steering angle: 0.8999332386013036 (pred)	-10.08 (actual)
Steering angle: -0.10856582350681962 (pred)	-10.08 (actual)
Steering angle: -0.5327782340362665 (pred)	-10.08 (actual)
Steering angle: -0.7450606735228215 (pred)	-10.08 (actual)
Steerin

# Summary(Conclusion):

1. Initially we have Dataset of  front dash camera recorded sequence of images and corresponding angles
2. so train the model using above dataset and our aim to predict sequence of steering angles
3. so initially we have 2.2GB datset provided by sullychen wich is 25 minjuts recorded video
4. our dataset is data.txt then we have load the data and splitted into train and validation then we loaded the data Batch wise 
5. we have performed mean baseline model  then we got MSE<0.19
6. then we have build end0-end CNN model which is input 66x200x3 for eacg image(each pixel divided by 0-255 after  the noermalized we created 5-CNN layers  then flatten layer at output  layer we applied sigmoid function 
7. then we have trained the model after train the model we store that model(in model.ckpt formated) into other folder.
8. then we did run the model using opencv then we could observe steering image angle rotaion for each image.
9. as per assignment we have changed  data split percentage(70%train and 30%test), adam optimezer with 1e(power)-3, we have changed dropout rate 0.5 and at final layer we have changed sigmoid activation finction
10. then we got output there is no any angle rotation in steering_wheele image.